<a href="https://colab.research.google.com/github/ganesh-deshmukh/Google-colab-hcr/blob/master/demoHCR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import tensorflow as tf
print(tf.__version__)

1.13.0-rc1


In [0]:
from tensorflow.examples.tutorials.mnist import input_data

In [4]:
mnist = input_data.read_data_sets("/MNIST_data/", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting /MNIST_data/t10k-images-idx3-ubyte.gz
Extracting /MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [0]:

n_train = mnist.train.num_examples
n_validation = mnist.validation.num_examples
n_test = mnist.test.num_examples

In [0]:
n_input = 784   # input layer (28x28 pixels)
n_hidden1 = 512 # 1st hidden layer
n_hidden2 = 256 # 2nd hidden layer
n_hidden3 = 128 # 3rd hidden layer
n_output = 10   # output layer (0-9 digits)

In [0]:
learning_rate = 0.01
n_iterations = 1000
batch_size = 128
dropout = 0.5

In [0]:
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_output])
keep_prob = tf.placeholder(tf.float32) 

In [9]:
weights = {
    'w1': tf.Variable(tf.truncated_normal([n_input, n_hidden1], stddev=0.1)),
    'w2': tf.Variable(tf.truncated_normal([n_hidden1, n_hidden2], stddev=0.1)),
    'w3': tf.Variable(tf.truncated_normal([n_hidden2, n_hidden3], stddev=0.1)),
    'out': tf.Variable(tf.truncated_normal([n_hidden3, n_output], stddev=0.1)),
}

Instructions for updating:
Colocations handled automatically by placer.


In [0]:
biases = {
    'b1': tf.Variable(tf.constant(0.1, shape=[n_hidden1])),
    'b2': tf.Variable(tf.constant(0.1, shape=[n_hidden2])),
    'b3': tf.Variable(tf.constant(0.1, shape=[n_hidden3])),
    'out': tf.Variable(tf.constant(0.1, shape=[n_output]))
}

In [11]:

layer_1 = tf.add(tf.matmul(X, weights['w1']), biases['b1'])
layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
layer_3 = tf.add(tf.matmul(layer_2, weights['w3']), biases['b3'])
layer_drop = tf.nn.dropout(layer_3, keep_prob)
output_layer = tf.matmul(layer_3, weights['out']) + biases['out']

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [12]:
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y, logits=output_layer))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [0]:
correct_pred = tf.equal(tf.argmax(output_layer, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [0]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

In [15]:
# train on mini batches
for i in range(n_iterations):
    batch_x, batch_y = mnist.train.next_batch(batch_size)
    sess.run(train_step, feed_dict={X: batch_x, Y: batch_y, keep_prob:dropout})

    # print loss and accuracy (per minibatch)
    if i%100==0:
        minibatch_loss, minibatch_accuracy = sess.run([cross_entropy, accuracy], feed_dict={X: batch_x, Y: batch_y, keep_prob:1.0})
        print("Iteration", str(i), "\t| Loss =", str(minibatch_loss), "\t| Accuracy =", str(minibatch_accuracy))

Iteration 0 	| Loss = 3.5732574 	| Accuracy = 0.09375
Iteration 100 	| Loss = 0.46837243 	| Accuracy = 0.8515625
Iteration 200 	| Loss = 0.28616896 	| Accuracy = 0.90625
Iteration 300 	| Loss = 0.27087328 	| Accuracy = 0.9140625
Iteration 400 	| Loss = 0.36240587 	| Accuracy = 0.90625
Iteration 500 	| Loss = 0.37170303 	| Accuracy = 0.9140625
Iteration 600 	| Loss = 0.24258243 	| Accuracy = 0.9296875
Iteration 700 	| Loss = 0.38374692 	| Accuracy = 0.9140625
Iteration 800 	| Loss = 0.24687342 	| Accuracy = 0.921875
Iteration 900 	| Loss = 0.5303529 	| Accuracy = 0.875


In [16]:
test_accuracy = sess.run(accuracy, feed_dict={X: mnist.test.images, Y: mnist.test.labels, keep_prob:1.0})
print("\nAccuracy on test set:", test_accuracy)


Accuracy on test set: 0.9172


In [58]:
import numpy as np
from PIL import Image


img = np.invert(Image.open("test_img.png").convert('L')).ravel()
prediction = sess.run(tf.argmax(output_layer,1), feed_dict={X: [img]})
print ("Prediction for test image:", np.squeeze(prediction))

Prediction for test image: 2


In [57]:
import numpy as np
from PIL import Image
# print("Image.open",type(Image.open("test_img.png")))
img = Image.open("six.png")
image = img.resize((28,28),Image.ANTIALIAS)
img2 = np.invert(image).ravel()
print("img is ",(image.size))
# print(img.shape)
# print(Image.fromarray(image).shape)
# # prediction = sess.run(tf.argmax(output_layer,1), feed_dict={X: [img]})
# # print ("Prediction for test image:", np.squeeze(prediction))

img is  (28, 28)


In [20]:
`
import numpy as np
import cv2
from PIL import Image
from matplotlib import pyplot as plt

cv_img = cv2.imread("six.png")
img2 = cv2.resize(cv_img, (28,28))
gray = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
inverted = cv2.bitwise_not(gray)


# print(cv_img.size)
# print(img2.size)
# print(gray.shape)
# print(inverted.ravel().shape)

# cv2.imshow("img2",img2)
# print(type(cv_img))
# print(type(img2))
# print(type(gray))

# print("\ncv_img" , np.invert(cv_img).ravel())
# print("\nimg2", img2)
# print("\ngray is =\n", gray)
# print("\ninverted is =\n", inverted)
# print(inverted)

predict = sess.run(tf.argmax(output_layer,1), feed_dict={X: [gray.ravel()]})
print ("Prediction for gray image:", np.squeeze(predict))

predict = sess.run(tf.argmax(output_layer,1), feed_dict={X: [inverted.ravel()]})
print ("Prediction for inverted image:", np.squeeze(predict))


# img3 = np.invert(img2).convert('L')).ravel()
# print("ravel is \n", np.invert(img2).ravel())                 
# prediction = sess.run(tf.argmax(output_layer,1), feed_dict={X: [img2.ravel()]})
# print ("Prediction for test image:", np.squeeze(prediction))                



Prediction for gray image: 2
Prediction for inverted image: 3
